In [8]:
import re
from pathlib import Path
from pprint import pprint

import llm_extract.azure_client as azure_client
import llm_extract.pydantic_extraction as pydantic_extraction
import requests
from pydantic import BaseModel, Field
from pypdf import PdfReader


def read_pdf(pdf_path: Path) -> str:
    pdf = PdfReader(str(pdf_path))
    return " ".join(page.extract_text() for page in pdf.pages)


def remove_watermark(text: str) -> str:
    """Remove the watermark from the text"""
    return re.sub(r"\d www\.andersenstories\.com", "", text)

def clean_text(text: str) -> str:
    """remove newlines and replace them with spaces"""
    return remove_watermark(text.replace("\n", " "))

def download_pdf(url: str, save_path: Path | None = None) -> Path:
    # If filename is not provided, extract it from the URL
    if not save_path:
        save_path = Path(url.split('/')[-1])
    if save_path.exists():
        return save_path
    response = requests.get(url)
    response.raise_for_status()  # Ensures that a valid response was received
    save_path.write_bytes(response.content)
    return save_path

# Data extraction med OpenAI function calling
Velkomment til denne tekniske workshop om data extraction med OpenAI function calling. I denne notebook vil vi få styr på følgende: 

1. Opsætning af Azure OpenAI (deployment, setup af API nøgler mv.)
2. Intro til (Azure) OpenAI API og SDK
3. Struktureret data extraction med function calls
4. Bedre function calls med Pydantic
5. Fri leg!

I denne notebook vil vi fokusere på eventyret "Kejserens nye klæder" af H.C. Andersen, en klassisk fortælling, der har begejstret og undret læsere i generationer. Let's get cracking!

## 1. Opsætning af Azure OpenAI
Lad os hopppe ind i [oai.azure.com](https://oai.azure.com/) og tjekke det ud!

## 2. Intro til (Azure) OpenAI API og SDK
Vi bruger [OpenAI's API](https://platform.openai.com/docs/api-reference/chat) og [Python SDK](https://github.com/openai/openai-python). For at forbinde til ens egen model, er det vigtigt at sætte en række miljøvariabler. Et eksempel kan ses i [`.template-env`](../.template-env). 

Det er også vigtigt at have installeret `openai` pakken. Det kan gøres med `pip install openai`. I dette repo bruger vi `poetry`. Alle nødvendige pakker kan installeres med `poetry install`. Lad os se det i aktion!

In [4]:
from openai import AzureOpenAI # importer clienten 
import os
from dotenv import load_dotenv

load_dotenv() # importerer .env filen

client = AzureOpenAI(
        api_key=os.getenv("OPENAI_API_KEY"),
        api_version=os.getenv("OPENAI_API_VERSION"),
        azure_endpoint=os.getenv("OPENAI_API_BASE"),
    )

Nu hvor vi har en client, kan vi prøve at generere et svar. Man skal bruge en model og en liste af "beskeder". En besked er et simpelt objekt med en `role` (enten `system`, `user`, eller `assistant`) og noget `content` (en string med prompten). Man skal give en liste af den fulde historik man vil medtage. Lad os prøve at snakke om kejserens nye klæder!

In [66]:
system_message = {"role": "system", "content": "You are an expert on H.C Andersen. You can answer all questions about all of his fairytales."}
question = {"role": "user", "content": "Opsummer eventyret om kejserens nye klæder i en præcis sætning. Nævn også de vigtigste karakterer i eventyret."}

response = client.chat.completions.create(
    model="gpt4", # NB: Det skal være det navn du har givet modellen i Azure!
    messages=[system_message, question],
)

Lad os tage et kig på svaret!

In [63]:
print(response)

ChatCompletion(id='chatcmpl-8cYWvvhsM7h6gBXaegJxFnq7gGU3M', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"Kejserens Nye Klæder" er et eventyr om en forfængelig kejser, der bliver narret af to svindlere til at tro, at han har købt en usynlig kappe, hvilket bliver afsløret af et barn, der påpeger at kejseren er nøgen.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'low'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1704199889, model='gpt-4', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=79, prompt_tokens=56, total_tokens=135), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'saf

Som man kan se er der en del attributer i svaret. Som sådan er vi kun interesseret i `choices` og det første element i listen, hvilket indeholder vores selve outputtet. I dette skal vi finde `message` og `content`. Alt det andet giver information om hvordan svaret er genereret, hvor mange tokens der er brugt, id'er osv. 

In [65]:
from pprint import pprint

pprint(response.choices[0].message.content)

('"Kejserens Nye Klæder" er et eventyr om en forfængelig kejser, der bliver '
 'narret af to svindlere til at tro, at han har købt en usynlig kappe, hvilket '
 'bliver afsløret af et barn, der påpeger at kejseren er nøgen.')


Ikke helt dårligt! Dog har vi ikke helt styr på outputtet, og hvor det kommer fra. Til det skal vi bruge `function calls` - lad os se det i aktion!

## 3. Struktureret data extraction med function calls
Function calls er en teknik fra OpenAI, der kan bruges til at få LLMs til at output JSON data. Man kan læse mere på deres documentation [her](https://platform.openai.com/docs/guides/function-calling).

Lad os sige, at vi vil have en funktion, der kan tage et eventyr og returnere en række informationer om det. Vi vil gerne have titlen, forfatteren, en liste af vigtige karakterer og en kort beskrivelse af eventyret. Lad os prøve at lave en function call, der kan gøre det!

```json
{'name': 'extract_fairy_tale_information', 
 'description': 'Extract high-level information about a fairy tale',
 'parameters': {'properties': {'story_title': {'title': 'Title of the fairy tale',
    'type': 'string'},
   'author_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'default': None,
    'title': 'Author of the fairy tale'},
   'important_characters': {'items': {'type': 'string'},
    'title': 'Important characters in the fairy tale',
    'type': 'array'},
   'one_sentence_summary': {'title': 'One sentence summary of the fairy tale',
    'type': 'string'}},
  'required': ['story_title', 'important_characters', 'one_sentence_summary'],
  'type': 'object'}}
```

Ovenfor kan man se en JSON beskrivelse af vores function call. Den indeholder en række informationer om funktionen, herunder navn, beskrivelse og parametre. Hver parameter har en række informationer, herunder titel, type og default værdi.



In [10]:
# Step 1: Download fairytale
URL = "https://www.andersenstories.com/da/andersen_fortaellinger/pdf/kejserens_nye_klaeder.pdf"


pdf_path = download_pdf(URL)
text = clean_text(read_pdf(pdf_path))
pprint(text[:200])

('Kejserens nye klæder For mange år siden levede en kejser, som holdt så uhyre '
 'meget af smukke nye klæder, at han gav alle sine penge ud for ret at blive '
 'pyntet. Han brød sig ikke om sine soldater, brød')


In [12]:
client = azure_client.initialize_client()

SYSTEM_MESSAGE = {"role": "system", "content": "You are an expert on H.C Andersen. You extract relevant information from a given fairy tale."}

function_call = {"name": "extract_fairy_tale_information", 
 'description': 'Extract high-level information about a fairy tale',
 'parameters': {'properties': {'story_title': {'title': 'Title of the fairy tale',
    'type': 'string'},
   'author_name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'default': "null",
    'title': 'Author of the fairy tale'},
   'important_characters': {'items': {'type': 'string'},
    'title': 'Important characters in the fairy tale',
    'type': 'array'},
   'one_sentence_summary': {'title': 'One sentence summary of the fairy tale',
    'type': 'string'}},
  'required': ['story_title', 'important_characters', 'one_sentence_summary'],
  'type': 'object'}}


completion = client.chat.completions.create(
  model="gpt4",
  messages=[SYSTEM_MESSAGE, {"role": "user", "content": text}],
        functions=[function_call],
        function_call={
            "name": function_call["name"],
        },
)


In [18]:
import pprint
print(f"{completion=}")
completion_output = completion.choices[0].message.function_call.arguments
pprint.pprint(f"{completion_output=}")

completion=ChatCompletion(id='chatcmpl-8dbZTv1RoYgkYxffJzhtxgjoExBnq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"story_title": "Kejserens nye Klæder",\n"author_name": "H.C Andersen",\n"important_characters": ["Kejseren", "To bedragere", "Minister", "Embedsmand", "Lille barn"],\n"one_sentence_summary": "En kejser, der elsker nye klæder, bliver narret af bedragere, der påstår at lave en usynlig kjole, og hans løgn bliver afsløret i en offentlig procession af et lille barn."\n}', name='extract_fairy_tale_information'), tool_calls=None), content_filter_results={})], created=1704449907, model='gpt-4', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=119, prompt_tokens=2811, total_tokens=2930), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'low'}, 'self_harm': {'fil

Svaret er fint, men det kan være svært at finde rundt i JSON kaldet. Heldigvis er der en mere pythonic løsning: Pydantic!

## 4. Bedre function calls med Pydantic

Præcis den samme function call kan laves med Pydantic. Lad os se det i aktion!

In [19]:
class extract_fairy_tale_information(BaseModel):
    """Extract high-level information about a fairy tale"""
    story_title: str = Field(..., title="Title of the fairy tale")
    author_name: str | None = Field(None, title="Author of the fairy tale")
    important_characters: list[str] = Field(..., title="Important characters in the fairy tale")
    one_sentence_summary: str = Field(..., title="One sentence summary of the fairy tale")
    

In [20]:
simple_information = pydantic_extraction.extract_with_schema(client=client, document=text, schema=extract_fairy_tale_information)

In [22]:
pprint.pprint(simple_information)

extract_fairy_tale_information(story_title='Kejserens nye klæder', author_name=None, important_characters=['Kejseren', 'to bedragere', 'minister', 'embedsmand', 'lille barn'], one_sentence_summary='En kejser bliver narret af to bedragere til at tro, at han har fået en usynlig dragt, og alle i hans rige er for bange for at indrømme, at de ikke kan se den, indtil et lille barn påpeger, at kejseren ikke har noget på.')


Dataen er næsten det samme, men det er blevet lidt nemmere at læse og arbejde med. Pydantic kan også bruges til at lave mere komplekse function calls med nestede data objekter. Fx. Hvis man skal lave en tidslinje over events:

In [44]:
class FairytaleEvent(BaseModel):
    """A significant plot point in a fairy tale"""
    name: str = Field(..., description="A descriptive title of the event")
    summary: str = Field(..., description="A one-sentence summary of the event")
    location: str = Field(..., description="The location where the event takes place")
    important_characters: list[str] = Field(..., description="The names of the characters involved in the event")

class extract_events_from_fairy_tale(BaseModel):
    """All the important plot points"""
    fairy_title: str = Field(..., description="The title of the fairy tale")
    events: list[FairytaleEvent] = Field(..., description="The most important plot points in the fairy tale. There should be at least 3 events per story")


In [45]:
initial_extraction = pydantic_extraction.extract_with_schema(client=client, document=text, schema=extract_events_from_fairy_tale)

In [49]:
pydantic_extraction.schema_to_function(extract_events_from_fairy_tale)

{'name': 'extract_events_from_fairy_tale',
 'description': 'All the important plot points',
 'parameters': {'$defs': {'FairytaleEvent': {'description': 'A significant plot point in a fairy tale',
    'properties': {'name': {'description': 'A descriptive title of the event',
      'title': 'Name',
      'type': 'string'},
     'summary': {'description': 'A one-sentence summary of the event',
      'title': 'Summary',
      'type': 'string'},
     'location': {'description': 'The location where the event takes place',
      'title': 'Location',
      'type': 'string'},
     'important_characters': {'description': 'The names of the characters involved in the event',
      'items': {'type': 'string'},
      'title': 'Important Characters',
      'type': 'array'}},
    'required': ['name', 'summary', 'location', 'important_characters'],
    'title': 'FairytaleEvent',
    'type': 'object'}},
  'properties': {'fairy_title': {'description': 'The title of the fairy tale',
    'title': 'Fairy Ti

In [48]:
for event in initial_extraction.events:
    pprint(event.dict())

{'important_characters': ['Kejseren'],
 'location': 'Kejserens palads',
 'name': 'Kejserens fascination for klæder',
 'summary': 'Kejseren er så besat af smukke nye klæder, at han bruger alle '
            'sine penge på at blive pyntet. Han har en kjole for hver time på '
            'dagen, og han er altid i garderoben.'}
{'important_characters': ['Kejseren', 'Bedragerne'],
 'location': 'Kejserens palads',
 'name': 'Bedragernes ankomst',
 'summary': 'To bedragere ankommer til byen og påstår, at de kan væve det '
            'smukkeste tøj, der bliver usynligt for enhver, der er dum eller '
            'ikke passer til sit embede. Kejseren betaler dem for at begynde '
            'at væve dette tøj.'}
{'important_characters': ['Kejseren', 'Embedsmænd', 'Bedragerne'],
 'location': 'Kejserens palads',
 'name': 'Kejserens embedsmænd kan ikke se tøjet',
 'summary': 'Kejseren sender sine embedsmænd for at se tøjet. De kan ikke se '
            'noget, men de lader som om, de kan for at und

C:\Users\JonathanHvithamarRys\AppData\Local\Temp\ipykernel_21800\3208738577.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  pprint(event.dict())


## 5. Fri leg!
- [ ] Find et dokument, du gerne vil lave en function call på. Det kan enten være [et andet eventyr](https://www.andersenstories.com/da/andersen_fortaellinger/list), eller et andet dokument.
- [ ] Lav en simpel function call, der kan lave en kort beskrivelse af dokumentet.
- [ ] Lav en nested function call, der kan lave en tidslinje over events i dokumentet.